# Actividad Integradora

## Descripción del problema

¡Felicidades! Eres el orgulloso propietario de 5 robots nuevos y un almacén lleno de cajas. El dueño anterior del almacén lo dejó en completo desorden, por lo que depende de tus robots organizar las cajas en algo parecido al orden y convertirlo en un negocio exitoso. 

Cada  robot  está  equipado  con  ruedas  omnidireccionales  y,  por  lo  tanto,  puede  conducir  en  las cuatro   direcciones.   Pueden   recoger   cajas   en   celdas   de   cuadrícula   adyacentes   con   sus manipuladores, luego llevarlas a otra ubicación e incluso construir pilas de hasta cinco cajas. Todos los robots están equipados con la tecnología de sensores más nueva que les permite recibir datos de sensores de las cuatro celdas adyacentes. Por tanto, es fácil distinguir si un campo está libre, es una pared, contiene una pila de cajas(y cuantas cajas hay en la pila)o está ocupado por otro robot. Los robots también tienen sensores de presión equipados que les indican si llevan una caja en ese momento. 

Lamentablemente, tu  presupuesto  resultó  insuficiente  para  adquirirun  software  de  gestión  de agentes múltiples de última generación. Pero eso no debería ser un gran problema ... ¿verdad? Tu tarea es enseñar a sus robots cómo ordenar su almacén. La organización delosagentesdepende de ti, siempre que todas las cajas terminen en pilas ordenadas de cinco.

Realiza la siguiente simulación:
* Inicializa las posiciones iniciales de las Kcajas. Todas las cajas están a nivel de piso, es decir, no hay pilas de cajas.
* Todos los agentes empiezan en posición aleatorias vacías.
* Se ejecuta el tiempo máximo establecido.

Deberás recopilar la siguiente información durante la ejecución:

* Tiempo necesario hasta que todas las cajas están en pilas de máximo 5 cajas.
* Número de movimientos realizados por todos los robots.
* Analiza si existe una estrategia que podría disminuir el tiempo dedicado, así como la cantidad de movimientos realizados. ¿Cómo sería? Descríbela.

In [1]:
!pip3 install mesa
!pip3 install numpy
!pip3 install matplotlib

In [2]:
from mesa import Agent, Model 
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd
import random

import time
import datetime

In [3]:
def obtener_almacen(modelo):
    almacen = np.zeros((modelo.grid.width, modelo.grid.height))
    for celda in modelo.grid.coord_iter():
        contenido_celda, x, y = celda
        for contenido in contenido_celda:
            if isinstance(contenido, Robot):
                almacen[x][y] = 6
            elif isinstance(contenido, Caja):
                almacen[x][y] = contenido.stack_size
    return almacen
    
class Robot(Agent):
    
    def __init__(self, id_unico, modelo):
        super().__init__(id_unico, modelo)
        self.nueva_posicion = None
        self.caja = None
        self.caja_coordenadas = None
        self.movimientos = 0
        
    def step(self):
        vecinos = self.model.grid.get_neighbors(
            self.pos,
            moore=False,
            include_center=False)
        # Defino el siguiente estado que va a tener el piso para la siguiente iteracion sin asignarlo todavia eso lo hago en el método `advance`.
        found_box = False
        self.nueva_posicion = self.pos
        if self.caja == None or self.caja_coordenadas == None:
            for vecino in vecinos:
                if isinstance(vecino, Caja):
                    vecino.siguiente_stack_size = vecino.stack_size
                    if vecino.siguiente_stack_size > 0 and vecino.siguiente_stack_size < 5 and self.caja_coordenadas != None and self.caja_coordenadas != vecino.pos:
                        self.caja = 1
                        vecino.siguiente_stack_size -= 1
                        self.nueva_posicion = self.pos
                        found_box = True
                        break
                    elif self.caja_coordenadas == None and vecino.siguiente_stack_size < 5 and vecino.siguiente_stack_size != 0:
                        self.caja_coordenadas = vecino.pos
                        self.nueva_posicion = self.pos
                        found_box = True
                        break
            if not found_box:
                vecindario = self.model.grid.get_neighbors(
                    self.pos,
                    moore=False,
                    include_center=False)
                temp_list = []
                for vecino2 in vecindario:
                    if len(self.model.grid.get_cell_list_contents(vecino2.pos)) == 1:
                        temp_list.append(vecino2.pos)
                if len(temp_list) != 0:
                    nueva_posicion = self.random.choice(temp_list)
                    self.nueva_posicion = nueva_posicion
        else:
            x2, y2 = self.caja_coordenadas
            x1, y1 = self.pos
            x = (x2 - x1)
            y = (y2 - y1)
            if abs(x) + abs(y) == 1:
                temp_cell_list = self.model.grid.get_cell_list_contents(self.caja_coordenadas)
                vecino5 = temp_cell_list[0]
                if vecino5.stack_size < 5:
                    vecino5.siguiente_stack_size = vecino5.stack_size + 1
                    self.caja = None
                    if vecino5.siguiente_stack_size == 5:
                        self.caja_coordenadas = None
                else:
                    vecinos = self.model.grid.get_neighbors(
                        self.pos,
                        moore=False,
                        include_center=False)
                    nueva_caja_pos = None
                    for vecino3 in vecinos:
                        temp_cell_list2 = self.model.grid.get_cell_list_contents(vecino3.pos)
                        if len(temp_cell_list2) == 1 and isinstance(vecino3, Caja) and vecino3.stack_size < 5:
                            vecino3.siguiente_stack_size = vecino3.stack_size + 1
                            self.caja = None
                            if vecino3.siguiente_stack_size != 5:
                                nueva_caja_pos = vecino3.pos
                            break
                    self.caja_coordenadas = nueva_caja_pos
            else:
                temp_pos = []
                if abs(x) > 0:
                    x = int(x / np.linalg.norm(x))
                    temp_pos.append((x1 + x, y1))
                if abs(y) > 0:
                    y = int(y / np.linalg.norm(y))
                    temp_pos.append((x1, y1 + y))
                self.nueva_posicion = self.pos
                i = self.random.choice(temp_pos)
                temp_cell_list = self.model.grid.get_cell_list_contents(i)
                if len(temp_cell_list) == 1:  
                    self.nueva_posicion = i
                else:
                    vecindario = self.model.grid.get_neighbors(
                        self.pos,
                        moore=False,
                        include_center=False)
                    temp_list = []
                    for vecino8 in vecindario:
                        if len(self.model.grid.get_cell_list_contents(vecino8.pos)) != 2:
                            temp_list.append(vecino8.pos)
                    if len(temp_list) != 0:
                        nueva_posicion = self.random.choice(temp_list)
                        self.nueva_posicion = nueva_posicion
        self.advance()
         
    def advance(self):
        # Actualizamos el estado del piso
        vecinos = self.model.grid.get_neighbors(
            self.pos,
            moore=False,
            include_center=False)
        
        for vecino in vecinos:
            if isinstance(vecino, Caja) and vecino.siguiente_stack_size != None: 
                vecino.stack_size = vecino.siguiente_stack_size
                vecino.siguiente_stack_size = None
        
        if self.pos != self.nueva_posicion:
            self.movimientos = self.movimientos + 1
            
        # Movemos la aspiradora a su nueva posicion
        self.model.grid.move_agent(self, self.nueva_posicion)
    
class Caja(Agent):
    # Cantidad máxima de cajas 5
    
    def __init__(self, pos, modelo, stack_size=0):
        super().__init__(pos, modelo)
        self.x, self.y = pos
        self.stack_size = stack_size
        self.siguiente_stack_size = None

class Almacen(Model):
    
    def __init__(self, m, n, num_agentes, num_cajas):
        self.num_agentes = num_agentes
        self.num_cajas = num_cajas
        self.grid = MultiGrid(m, n, False)
        self.schedule = RandomActivation(self)
               
        # Posicionar cajas de forma aleatoria
        lista_celdas_sin_cajas = list(self.grid.empties)
        for celdas in range(num_cajas):
            celda_vacia = random.choice(lista_celdas_sin_cajas)
            caja = Caja(celda_vacia, self)
            caja.stack_size = 1
            self.grid.place_agent(caja, celda_vacia)
            self.schedule.add(caja)
            lista_celdas_sin_cajas.remove(celda_vacia)
        
        # Posicionar celdas sin cajas
        lista_celdas_sin_cajas = list(self.grid.empties)
        for celdas in lista_celdas_sin_cajas:
            caja = Caja(celdas, self)
            self.grid.place_agent(caja, celdas)
            self.schedule.add(caja)
        
        # Posicionar agentes robots
        for i in range(num_agentes):
            celda_vacia = random.choice(lista_celdas_sin_cajas)
            robot = Robot(i, self)
            self.grid.place_agent(robot, celda_vacia)
            self.schedule.add(robot)
            lista_celdas_sin_cajas.remove(celda_vacia)
            
        self.colectordatos = DataCollector(
            model_reporters={'Almacen': obtener_almacen},
            agent_reporters={'Movimientos': lambda a: getattr(a, 'movimientos', None)}
        )
    
    def step(self):
        self.colectordatos.collect(self)
        self.schedule.step()    
    
    def todasCeldasOrdenadas(self):
        celdas_ordenadas = 0
        for celda in self.grid.coord_iter():
            contenido_celda, x, y = celda
            for contenido in contenido_celda:
                if isinstance(contenido, Caja) and contenido.stack_size == 5:
                    celdas_ordenadas += 1
        if celdas_ordenadas == (self.num_cajas / 5):
            return True
        else:
            return False

In [4]:
# Datos de la habitacion:
M = 8
N = 8

# Numero de agentes
NUM_ROBOTS = 5
NUM_CAJAS = 25

# Tiempo máximo de ejecución (segundos)
TIEMPO_MAXIMO_EJECUCION = 0.1

start_time = time.time()
tiempo_inicio = str(datetime.timedelta(seconds=TIEMPO_MAXIMO_EJECUCION))
modelo = Almacen(M, N, NUM_ROBOTS, NUM_CAJAS)

while((time.time() - start_time) < TIEMPO_MAXIMO_EJECUCION and not modelo.todasCeldasOrdenadas()):
    modelo.step()
    #print(str(datetime.timedelta(seconds=(time.time() - start_time))))

# Imprimimos el tiempo que le tomó correr al modelo.
tiempo_ejecucion = str(datetime.timedelta(seconds=(time.time() - start_time)))

[<__main__.Caja object at 0x7f8888153950>, <__main__.Caja object at 0x7f8888153cd0>, <__main__.Caja object at 0x7f8888153050>, <__main__.Caja object at 0x7f8888153910>]
[<__main__.Caja object at 0x7f8888153950>, <__main__.Caja object at 0x7f8888153cd0>, <__main__.Caja object at 0x7f8888153050>, <__main__.Caja object at 0x7f8888153910>]
[<__main__.Caja object at 0x7f8888153950>, <__main__.Caja object at 0x7f8888153cd0>, <__main__.Caja object at 0x7f8888153050>, <__main__.Caja object at 0x7f8888153910>]
[<__main__.Caja object at 0x7f8888153950>, <__main__.Caja object at 0x7f8888153cd0>, <__main__.Caja object at 0x7f8888153050>, <__main__.Caja object at 0x7f8888153910>]
[<__main__.Caja object at 0x7f8888151150>, <__main__.Caja object at 0x7f8888153450>, <__main__.Caja object at 0x7f88881512d0>, <__main__.Caja object at 0x7f8888151e50>]
[<__main__.Caja object at 0x7f8888151150>, <__main__.Caja object at 0x7f8888153450>, <__main__.Caja object at 0x7f88881512d0>, <__main__.Caja object at 0x7

## Visualización

In [5]:
todas_celdas = modelo.colectordatos.get_model_vars_dataframe()

In [6]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(todas_celdas.iloc[0][0], cmap='Purples')

def animate(i):
    patch.set_data(todas_celdas.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(todas_celdas))

In [7]:
anim

In [8]:
movimientos = modelo.colectordatos.get_agent_vars_dataframe()

print('Tiempo necesario hasta que todas las celdas estén limpias:', tiempo_ejecucion, '/', tiempo_inicio)
print('Número de movimientos realizados por todos los agentes:', movimientos.tail()['Movimientos'].sum())

Tiempo necesario hasta que todas las celdas estén limpias: 0:00:00.111547 / 0:00:00.100000
Número de movimientos realizados por todos los agentes: 27.0
